In [3]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

## Задание

### Есть 2 группы студентов одна из которых ходила на подготовительные курсы, другая нет.
### Нужно понять повлияло ли посещение курсов на балл студентов между группами.

In [4]:
student_after_score = pd.read_csv('E:/Downloads/student_after_score.csv')
student_before_score = pd.read_csv('E:/Downloads/student_before_score.csv')
student_group = pd.read_csv('E:/Downloads/student_group.csv')


In [10]:
for i in [student_after_score, student_before_score, student_group]: print(i.head(), end='\n\n')

   student_id  after_score
0           1           92
1           2           79
2           3           92
3           4           90
4           5           98

   student_id  before_score
0           1            75
1           2            81
2           3            58
3           4            47
4           5            76

   student_id      group
0           1  no_course
1           2     course
2           3     course
3           4     course
4           5     course



In [9]:
pd.concat([student_after_score, student_before_score]).describe()

,student_id,after_score,before_score
count,2000.000000,1000.000000,1000.00000
mean,500.500000,84.623000,63.92500
std,288.747186,8.641154,14.59851
min,1.000000,70.000000,40.00000
25%,250.750000,77.000000,51.00000
50%,500.500000,85.000000,64.00000
75%,750.250000,92.000000,77.00000
max,1000.000000,99.000000,89.00000


### Различие групп достаточно видимое, проверим количество тех, кто был на курсах, а кто нет.

In [40]:
student_group.group.value_counts(normalize=True).mul(1000)

no_course    512.0
course       488.0
Name: group, dtype: float64

In [35]:
student_group.query('group == "course"').shape

(488, 2)

### Выберем тех кто ходил и объединим группы с помощью merge по столбцу student_id

In [46]:
merged_groups = student_group.query('group == "course"').merge(student_after_score,on='student_id') \
                                                        .merge(student_before_score,on='student_id')
merged_groups.head()

,student_id,group,after_score,before_score
0,2,course,79,81
1,3,course,92,58
2,4,course,90,47
3,5,course,98,76
4,6,course,76,67


In [50]:
merged_groups.describe()

,student_id,after_score,before_score
count,488.000000,488.000000,488.000000
mean,484.760246,83.903689,63.301230
std,291.348817,8.532397,14.625841
min,2.000000,70.000000,40.000000
25%,235.750000,76.000000,51.000000
50%,482.500000,84.000000,63.500000
75%,743.250000,90.000000,75.000000
max,1000.000000,99.000000,89.000000


### Различие групп осталось таким же заметным, предположу, что среди тех кто не ходил на курсы различия будут примерно похожи с теми, кто сходил

### Теперь проверим нормальность распределений с помощью теста Шапиро-Уилка. Определим для того, чтобы определить какой тест в будущем нужно использовать для проверки на различие между этими выборками

In [65]:
st.shapiro(merged_groups['after_score'])

ShapiroResult(statistic=0.9546159505844116, pvalue=4.223854599416654e-11)

In [73]:
st.shapiro(merged_groups['before_score'])

ShapiroResult(statistic=0.9492893815040588, pvalue=6.953233128159653e-12)

### Оба распределения ненормальные потому что pvalue гораздо ниже уровня значимости в 0.05 и мы подтверждаем альтернативную гипотезу, что распределение ненормальное

### В таком случае используем для проверки тест Манна-Уитни

### Баллы это дискретные данные, поэтому учитываем поправку на непрерывность.
### И выбираем two-sided  чтобы проверить выборки между собой

In [80]:
st.mannwhitneyu(merged_groups.before_score, merged_groups.after_score, True, 'two-sided')

MannwhitneyuResult(statistic=30407.5, pvalue=3.23053961900692e-90)

### p-значение меньше уровня значимости в 0.05. Значит, что различие между выборками есть, даже с доверительным интервалом в 99%.
### Различие между выборками статистически значимо. А теперь узнаем, где именно различие между выборками статистически значимо через Бутстрэп, выберем среднее и медиану этих групп.

In [115]:
from scipy.stats import norm
def get_bootstrap(
    data_column_1, 
    data_column_2, 
    boot_it = 1000, 
    statistic = np.mean, 
    bootstrap_conf_level = 0.95 
):
    boot_len = max([len(data_column_1), len(data_column_2)])
    boot_data = []
    for i in range(boot_it): 
        samples_1 = data_column_1.sample(
            boot_len, 
            replace = True 
        ).values
        
        samples_2 = data_column_2.sample(
            boot_len, 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1-samples_2)) 
        
    pd_boot_data = pd.DataFrame(boot_data)
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
    
    return {"p_value": p_value}

In [129]:
print(f'Среднее выборок статистически значимо отличается  {get_bootstrap(x, y)}')

Среднее выборок статистически значимо отличается  {'p_value': 3.625152890514245e-153}


In [127]:
x =  merged_groups.before_score 
y = merged_groups.after_score
print(f'Медиана выборок статистически значимо отличается,  {get_bootstrap(x, y, statistic = np.median)}')

Бутстреп медиана выборок статистически значимо отличается,  {'p_value': 1.9728620218600353e-71}


### Помимо общего различия выборок среднее и медиана баллов студентов, которые ходили на курсы тоже статистически значимо отличается.